In [1]:
import pandas as pd
import numpy as np

In [2]:
!pip install aif360
!pip install BlackBoxAuditing
!pip install fairlearn==0.4.6

# Load and Split Data

In [3]:
from aif360.datasets import AdultDataset

/home/sophie/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sophie/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sophie/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sophie/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [4]:
ad = AdultDataset(instance_weights_name='fnlwgt', features_to_drop=[])

W0425 11:27:59.496087 140070176089920 standard_dataset.py:101] Missing Data: 3620 rows removed from AdultDataset.


In [5]:
ad_train, ad_test = ad.split(2)

# Unconstrained Model

In [6]:
from sklearn.linear_model import LogisticRegression
from copy import deepcopy
from aif360.metrics import ClassificationMetric

In [7]:
ad_df_train, ad_attrs_train = ad_train.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=True)
ad_df_test, ad_attrs_test = ad_test.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=True)

In [8]:
Y_train = np.array(ad_df_train[['income-per-year']]).reshape(((len(ad_df_train),)))
Xs_train = np.array(ad_df_train.drop(columns='income-per-year'))
Y_test = np.array(ad_df_test[['income-per-year']]).reshape(((len(ad_df_test),)))
Xs_test = np.array(ad_df_test.drop(columns='income-per-year'))

In [9]:
clf = LogisticRegression(max_iter = 300, solver='liblinear').fit(Xs_train, Y_train)

In [10]:
predicted_labels_train = clf.predict(Xs_train).reshape((len(Y_train), 1))
predicted_labels = clf.predict(Xs_test).reshape((len(Y_test), 1))

In [32]:
ad_pred = deepcopy(ad_test)
ad_pred_train = deepcopy(ad_train)
ad_pred.labels = predicted_labels
ad_pred_train.labels = predicted_labels_train

In [12]:
u = [{'race': 0}]
p = [{'race': 1}]
metrics = ClassificationMetric(ad_test,ad_pred,unprivileged_groups=u, privileged_groups=p)
test_acc = metrics.accuracy()
DI = metrics.disparate_impact()
EO = metrics.average_abs_odds_difference()
print('Original Test Accuracy:', test_acc)
print('Original Demographic Parity ratio:', DI)
print('Original Average Absolute Odds diff:', EO)

Original Test Accuracy: 0.8516964899300383
Original Demographic Parity ratio: 0.5652787213393603
Original Average Absolute Odds diff: 0.049887323350057494


# Demographic Parity (Independence)

## Feldman et al. Repair (Preprocessing)

In [13]:
import aif360.algorithms.preprocessing as AIF

In [14]:
repairer = AIF.DisparateImpactRemover(repair_level=1.0, sensitive_attribute='race')
repaired_train = repairer.fit_transform(ad_train)
repaired_test = repairer.fit_transform(ad_test)

In [15]:
repaired_df_train, repaired_attrs_train = repaired_train.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=True)
repaired_df_test, repaired_attrs_test = repaired_test.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=True)

In [16]:
Y_rep_train = np.array(repaired_df_train[['income-per-year']]).reshape(((len(repaired_df_train),)))
Xs_rep_train = np.array(repaired_df_train.drop(columns='income-per-year'))

Y_rep_test = np.array(repaired_df_test[['income-per-year']]).reshape(((len(repaired_df_test),)))
Xs_rep_test = np.array(repaired_df_test.drop(columns='income-per-year'))

In [17]:
clf_rep = LogisticRegression(max_iter = 300, solver='liblinear').fit(Xs_rep_train, Y_rep_train)

In [18]:
predicted_labels_rep = clf_rep.predict(Xs_rep_test).reshape((len(Y_rep_test), 1))

In [19]:
ad_pred_rep = deepcopy(ad_test)
ad_pred_rep.labels = predicted_labels_rep

In [20]:
metrics_rep = ClassificationMetric(ad_test,ad_pred_rep,unprivileged_groups=u, privileged_groups=p)
test_acc_rep = metrics_rep.accuracy()
DI_rep = metrics_rep.disparate_impact()
EO_rep = metrics_rep.average_abs_odds_difference()
print('Repair Test Accuracy:', test_acc_rep)
print('Repair Demographic Parity ratio:', DI_rep)
print('Repair Average Absolute Odds diff:', EO_rep)

Repair Test Accuracy: 0.8511096256954943
Repair Demographic Parity ratio: 0.5748155706381144
Repair Average Absolute Odds diff: 0.04189026077198628


## Kamishima et al. Regularization (Regularization)

In [21]:
!pip install tensorflow

In [22]:
from aif360.algorithms.inprocessing import PrejudiceRemover

In [47]:
PrejRemover = PrejudiceRemover(eta=0.1, sensitive_attr='race', class_attr='income-per-year')

In [48]:
PrejRemover = PrejRemover.fit(ad_train)
kamishima_pred = PrejRemover.predict(ad_test)

In [49]:
metrics_kamishima = ClassificationMetric(ad_test,kamishima_pred,unprivileged_groups=u, privileged_groups=p)
test_acc_prej = metrics_kamishima.accuracy()
DI_prej = metrics_kamishima.disparate_impact()
EO_prej = metrics_kamishima.average_abs_odds_difference()
print('Prej Remover Test Accuracy:', test_acc_prej)
print('Prej Remover Demographic Parity ratio:', DI_prej)
print('Prej Remover Average Absolute Odds diff:', EO_prej)

Prej Remover Test Accuracy: 0.8453421761768007
Prej Remover Demographic Parity ratio: 0.4978724331429025
Prej Remover Average Absolute Odds diff: 0.0756202852512517


## Agarwal et al (Reduction)

In [42]:
from aif360.algorithms.inprocessing.exponentiated_gradient_reduction import ExponentiatedGradientReduction

In [43]:
estimator = LogisticRegression(solver='liblinear')
np.random.seed(0) #need for reproducibility
exp_grad_red_dp = ExponentiatedGradientReduction(estimator=estimator, 
                                              constraints="DemographicParity",
                                              drop_prot_attr=False)

In [44]:
exp_grad_red_dp.fit(ad_train)
exp_grad_red_pred_dp = exp_grad_red_dp.predict(ad_test)

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


In [46]:
metrics_red_dp = ClassificationMetric(ad_test,exp_grad_red_pred_dp,unprivileged_groups=u, privileged_groups=p)
test_acc_red_dp = metrics_red_dp.accuracy()
DI_red_dp = metrics_red_dp.disparate_impact()
EO_red_dp = metrics_red_dp.average_abs_odds_difference()
print('DP Reduction Test Accuracy:', test_acc_red_dp)
print('DP Reduction Demographic Parity ratio:', DI_red_dp)
print('DP Reduction Average Absolute Odds diff:', EO_red_dp)

DP Reduction Test Accuracy: 0.8320405411045667
DP Reduction Demographic Parity ratio: 0.9469555766712154
DP Reduction Average Absolute Odds diff: 0.0630372657437408


# Equalized Odds (Separation)

## Hardt et al. (Postprocessing)

In [29]:
from aif360.algorithms.postprocessing import EqOddsPostprocessing

In [30]:
postprocessing = EqOddsPostprocessing(u, p)

In [33]:
postprocessing = postprocessing.fit(ad_train, ad_pred_train)
postprocess_pred = postprocessing.predict(ad_pred)

In [35]:
metrics_post = ClassificationMetric(ad_test,postprocess_pred,unprivileged_groups=u, privileged_groups=p)
test_acc_post = metrics_post.accuracy()
DI_post = metrics_post.disparate_impact()
EO_ppost = metrics_post.average_abs_odds_difference()
print('Postprocessing Test Accuracy:', test_acc_post)
print('Postprocessing Demographic Parity ratio:', DI_post)
print('Postprocessing Average Absolute Odds diff:', EO_ppost)

Postprocessing Test Accuracy: 0.8429309786202139
Postprocessing Demographic Parity ratio: 0.7083046158936913
Postprocessing Average Absolute Odds diff: 0.016684602600477202


## Agarwal et al (Reduction)

In [37]:
estimator = LogisticRegression(solver='liblinear')
np.random.seed(0) #need for reproducibility
exp_grad_red = ExponentiatedGradientReduction(estimator=estimator, 
                                              constraints="EqualizedOdds",
                                              drop_prot_attr=False)

In [38]:
exp_grad_red.fit(ad_train)
exp_grad_red_pred = exp_grad_red.predict(ad_test)

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


In [40]:
metrics_red_eo = ClassificationMetric(ad_test,exp_grad_red_pred,unprivileged_groups=u, privileged_groups=p)
test_acc_red_eo = metrics_red_eo.accuracy()
DI_red_eo = metrics_red_eo.disparate_impact()
EO_red_eo = metrics_red_eo.average_abs_odds_difference()
print('EO Reduction Test Accuracy:', test_acc_red_eo)
print('EO Reduction Demographic Parity ratio:', DI_red_eo)
print('EO Reduction Average Absolute Odds diff:', EO_red_eo)

EO Reduction Test Accuracy: 0.8395849432292414
EO Reduction Demographic Parity ratio: 0.6803765727104913
EO Reduction Average Absolute Odds diff: 0.009446494651760642
